In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import warnings

warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm

In [ ]:
# read final processed data
df = pd.read_csv('../inputs/final_processed_data.csv')
df.head()

In [ ]:
# Method to split final processed data in training / testing
def split_data_in_training_testing(df):

    data = df.sort_values(by=['goals_per_match_year3'])

    data = data.drop('name', 1)

    y = data[['goals_per_match_year3']].values.astype('float32')

    data = data.drop('goals_per_match_year3', 1)
    x = data.values.astype('float32')

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10)
    
    return x_train, x_test, y_train, y_test

In [ ]:
# Method to compute best model using k-fold validation
def compute_optimal_hyperparameters(x_train, y_train):
    
    alpha_list = [0.1, 0.2, 0.5, 1]
    fit_intercept_list = [False, True]
    normalize_list = [False, True]
    warm_start_list = [False, True]
    positive_list = [False, True]

    best_score = -1000
    
    for alpha in alpha_list:
        for fit_intercept in fit_intercept_list:
            for normalize in normalize_list:
                for warm_start in warm_start_list:
                    for positive in positive_list:

                        current_model = Lasso(alpha=alpha, fit_intercept=fit_intercept, normalize=normalize, 
                                              warm_start=warm_start, positive=positive)
                        current_score = cross_val_score(current_model, x_train, y_train, cv=5).mean()

                        if current_score > best_score:
                            best_score = current_score
                            best_model = current_model
    
    return best_score, best_model

In [ ]:
# Method to plot model accuracy on test data
def plot_model_accuracy_on_test_data(model, x_test, y_test):
    
    y_pred = model.predict(x_test)

    plt.plot(y_pred, label='prediction')
    plt.plot(y_test, label='true',alpha =0.3)
    plt.legend()
    plt.show() 

    fig, ax = plt.subplots()
    plt.title('Actual value vs predicted value (goals per match year 3)')
    ax.scatter(y_test, y_pred, edgecolors=(0, 0, 0),alpha =0.3)
    ax.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'k--', lw=4)
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    plt.show()

In [ ]:
# Method to compute best data split
def compute_best_data_split(df, iterations):
    
    best_test_score = -1
    
    for i in tqdm(range(0, iterations)):

        x_train, x_test, y_train, y_test = split_data_in_training_testing(df)

        validation_score, model = compute_optimal_hyperparameters(x_train, y_train)

        model.fit(x_train,y_train)

        test_score = model.score(x_test, y_test)
        
        if abs(validation_score - test_score) < 0.05 and test_score > best_test_score:
            
            best_model = model
            best_validation_score = validation_score
            best_test_score = test_score
            
    plot_model_accuracy_on_test_data(best_model, x_test, y_test)

    return best_model, best_validation_score, best_test_score

In [ ]:
model, validation_score, test_score = compute_best_data_split(df, 100)

print("Validation score:", validation_score)
print("Test score:", test_score)
print(model)

In [ ]:
model.coef_